In [1]:
!pip install -r ../../../requirements.txt
import nltk
nltk.download("stopwords")


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python3.118.exe -m pip install --upgrade pip
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
!pip freeze

aiohappyeyeballs==2.4.3
aiohttp==3.10.10
aiosignal==1.3.1
altair==5.4.1
annotated-types==0.7.0
anyio==4.6.2.post1
asttokens==2.4.1
attrs==24.2.0
bcrypt==4.2.0
blinker==1.8.2
cachetools==5.5.0
certifi==2024.8.30
charset-normalizer==3.4.0
click==8.1.7
colorama==0.4.6
comm==0.2.2
dataclasses-json==0.6.7
debugpy==1.8.7
decorator==5.1.1
distro==1.9.0
dnspython==2.7.0
ecdsa==0.19.0
email_validator==2.2.0
executing==2.1.0
fastapi==0.115.4
fastapi-cli==0.0.5
filelock==3.16.1
frozenlist==1.5.0
fsspec==2024.10.0
gitdb==4.0.11
GitPython==3.1.43
greenlet==3.1.1
h11==0.14.0
httpcore==1.0.6
httptools==0.6.4
httpx==0.27.2
huggingface-hub==0.26.2
idna==3.10
ipykernel==6.29.5
ipython==8.29.0
jedi==0.19.1
Jinja2==3.1.4
jiter==0.7.0
joblib==1.4.2
jsonpatch==1.33
jsonpointer==3.0.0
jsonschema==4.23.0
jsonschema-specifications==2024.10.1
jupyter_client==8.6.3
jupyter_core==5.7.2
langchain==0.3.4
langchain-community==0.3.3
langchain-core==0.3.15
langchain-text-splitters==0.3.2
langsmith==0.1.139
markdown-it

In [3]:
import pandas as pd
import openai
from utils import *
import utils
import constants
from sklearn.metrics.pairwise import cosine_similarity


c:\Users\ASUS\Desktop\thesius.ai\backend\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ASUS\Desktop\thesius.ai\backend\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
openai.api_key = constants.OPENAI_API_KEY

### Query S2 paper index

In [5]:
query = "How RAG works and how it impacts AI ?"

# json to pandas dataframe
search_results = search(preprocess_query(query))

if search_results['total']==0:
    print('No results found - Try another query')
else:
    df = pd.DataFrame(search_results['data']).dropna()

In [6]:
df.head()

,paperId,title,abstract,venue,year
0,9538df9a77549ca019b522eedfd5b367310293f7,Human intelligence and artificial intelligence...,This article acknowledges the profound transfo...,Journal on Innovation and Sustainability RISUS,2024
1,5d9cd911f6cefa87707bd5b360bd0f5b762b2749,Impact of Artificial Intelligence in the field...,Artificial intelligence (AI) has potentially t...,Journal of Physics: Conference Series,2021
2,2162d3b2588ff48b5046a6a9f144591800082ea6,AI-Driven Marketplaces and Price Prediction To...,This paper aims at identifying and analyzing t...,Babylonian Journal of Artificial Intelligence,2023
3,130b9d2ae352e9eacc6e4f048cfdc9ee277e9d46,Corporate Governance in the Digital Age: A Com...,This research examines the intersection of dis...,E3S Web of Conferences,2023
4,e8b18bdb2196fbfa50c4e56a8626a2e2bb168197,Empirically Understanding the Potential Impact...,"In the coming years, Artificial Intelligence (...",Proc. ACM Hum. Comput. Interact.,2024


### Re-ranking using SPECTER

In [7]:
# merge columns title and abstract into a string separated by tokenizer.sep_token and store it in a list

df['title_abs'] = [d['title'] + tokenizer.sep_token + (d.get('abstract') or '') for d in df.to_dict('records')]
df['n_tokens'] = df.title_abs.apply(lambda x: len(tokenizer.encode(x)))
df['title_abs'][2]

"AI-Driven Marketplaces and Price Prediction Tools for Rag Pickers: Enhancing Economic Opportunities in Africa's Circular Economy[SEP]This paper aims at identifying and analyzing the use of the AI solutions in enhancing the economic status of rag pickers within Africa’s circular economy sector and their access to markets. In this study, we explore how effectively the employment of AI technologies advances the welfare of the weaver people by identifying new potential methods of income generation and skill development in waste management techniques. Our research adopts a case study approach, examining the potential of two key AI applications: The resources include a marketplace that helps rag pickers find buyers for recyclable materials and price forecasting applications to aid selling. The role of such technologies in enhancing the efficiency of materials exchange, increasing or decreasing income, and improving rag pickers’ understanding of the market is discussed. The study entailed th

In [8]:
# get embeddings for each document and query

doc_embeddings = get_specter_embeddings(list(df['title_abs']))
query_embeddings = get_specter_embeddings(advanced_preprocess_query(query))

df['specter_embeddings'] = list(doc_embeddings)
df.head()

[+] processed query:  Query expanded random access group RAG work impact artificial intelligence AI


,paperId,title,abstract,venue,year,title_abs,n_tokens,specter_embeddings
0,9538df9a77549ca019b522eedfd5b367310293f7,Human intelligence and artificial intelligence...,This article acknowledges the profound transfo...,Journal on Innovation and Sustainability RISUS,2024,Human intelligence and artificial intelligence...,226,"[0.21166977, 0.3828036, -0.82473946, 0.2124952..."
1,5d9cd911f6cefa87707bd5b360bd0f5b762b2749,Impact of Artificial Intelligence in the field...,Artificial intelligence (AI) has potentially t...,Journal of Physics: Conference Series,2021,Impact of Artificial Intelligence in the field...,121,"[0.24786988, 0.2989162, -0.72270244, 0.2948508..."
2,2162d3b2588ff48b5046a6a9f144591800082ea6,AI-Driven Marketplaces and Price Prediction To...,This paper aims at identifying and analyzing t...,Babylonian Journal of Artificial Intelligence,2023,AI-Driven Marketplaces and Price Prediction To...,324,"[0.0011445875, 0.22520094, -0.6562357, -0.0665..."
3,130b9d2ae352e9eacc6e4f048cfdc9ee277e9d46,Corporate Governance in the Digital Age: A Com...,This research examines the intersection of dis...,E3S Web of Conferences,2023,Corporate Governance in the Digital Age: A Com...,275,"[0.3889911, -0.013424439, -0.86219573, 0.02648..."
4,e8b18bdb2196fbfa50c4e56a8626a2e2bb168197,Empirically Understanding the Potential Impact...,"In the coming years, Artificial Intelligence (...",Proc. ACM Hum. Comput. Interact.,2024,Empirically Understanding the Potential Impact...,306,"[0.2136501, 0.27276045, -0.93551767, 0.2673642..."


In [9]:
# find the cosine similarity between the query and the documents
df['similarity'] = cosine_similarity(query_embeddings, doc_embeddings).flatten()

# sort the dataframe by similarity
df.sort_values(by='similarity', ascending=False, inplace=True)
df.head()

,paperId,title,abstract,venue,year,title_abs,n_tokens,specter_embeddings,similarity
16,77179e5ff669452b9bea479a4236a6e2009ee422,The Power of Noise: Redefining Retrieval for R...,Retrieval-Augmented Generation (RAG) has recen...,Annual International ACM SIGIR Conference on R...,2024,The Power of Noise: Redefining Retrieval for R...,290,"[0.046463024, 0.43938, -0.7837731, -0.26589015...",0.923377
12,917cef7bf9841044af50f4bdd75db057aba1aed9,RAGProbe: An Automated Approach for Evaluating...,Retrieval Augmented Generation (RAG) is increa...,arXiv.org,2024,RAGProbe: An Automated Approach for Evaluating...,312,"[0.27762347, 0.38148606, -0.8177407, -0.270688...",0.902334
10,dcbcba4cf01b36e499060a7d4e9a00e808f4daf9,ML Based Solutions for Greenhouse Gas Emission...,This literature review will serve as the basis...,International Journal on Perceptive and Cognit...,2023,ML Based Solutions for Greenhouse Gas Emission...,226,"[0.22443536, 0.23313893, -0.8218236, 0.1431330...",0.895758
0,9538df9a77549ca019b522eedfd5b367310293f7,Human intelligence and artificial intelligence...,This article acknowledges the profound transfo...,Journal on Innovation and Sustainability RISUS,2024,Human intelligence and artificial intelligence...,226,"[0.21166977, 0.3828036, -0.82473946, 0.2124952...",0.893167
5,e9ad33f3579d61b4a8ad684edd03cb10df6a27d0,POSTHUMANISM: ALTERNATIVE REALITIES AND AI IN ...,The paper focuses on the science fiction novel...,SAKHAROV READINGS 2022: ENVIRONMENTAL PROBLEMS...,2022,POSTHUMANISM: ALTERNATIVE REALITIES AND AI IN ...,179,"[-0.11198905, 0.31442636, -0.8300437, -0.09335...",0.893007


## Get answer from GPT-3

In [14]:
!pip install -r ../../../requirements.txt


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python3.118.exe -m pip install --upgrade pip


In [10]:
# Define a function to generate a summary of each abstract
def summarize_abstract(abstract):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": f"Summarize the following abstract:\n\n{abstract}"}],
        max_tokens=50,
        temperature=0.7
    )
    summary = response.choices[0].message.content.strip()
    return summary



# Apply the functions to create the abstract_summary and relevance columns
df['abstract_summary'] = df['abstract'].apply(summarize_abstract)


# Display the updated DataFrame
df.head()



,paperId,title,abstract,venue,year,title_abs,n_tokens,specter_embeddings,similarity,abstract_summary
16,77179e5ff669452b9bea479a4236a6e2009ee422,The Power of Noise: Redefining Retrieval for R...,Retrieval-Augmented Generation (RAG) has recen...,Annual International ACM SIGIR Conference on R...,2024,The Power of Noise: Redefining Retrieval for R...,290,"[0.046463024, 0.43938, -0.7837731, -0.26589015...",0.923377,Retrieval-Augmented Generation (RAG) is a meth...
12,917cef7bf9841044af50f4bdd75db057aba1aed9,RAGProbe: An Automated Approach for Evaluating...,Retrieval Augmented Generation (RAG) is increa...,arXiv.org,2024,RAGProbe: An Automated Approach for Evaluating...,312,"[0.27762347, 0.38148606, -0.8177407, -0.270688...",0.902334,Retrieval Augmented Generation (RAG) is common...
10,dcbcba4cf01b36e499060a7d4e9a00e808f4daf9,ML Based Solutions for Greenhouse Gas Emission...,This literature review will serve as the basis...,International Journal on Perceptive and Cognit...,2023,ML Based Solutions for Greenhouse Gas Emission...,226,"[0.22443536, 0.23313893, -0.8218236, 0.1431330...",0.895758,This literature review discusses the analysis ...
0,9538df9a77549ca019b522eedfd5b367310293f7,Human intelligence and artificial intelligence...,This article acknowledges the profound transfo...,Journal on Innovation and Sustainability RISUS,2024,Human intelligence and artificial intelligence...,226,"[0.21166977, 0.3828036, -0.82473946, 0.2124952...",0.893167,This article discusses the impact of Artificia...
5,e9ad33f3579d61b4a8ad684edd03cb10df6a27d0,POSTHUMANISM: ALTERNATIVE REALITIES AND AI IN ...,The paper focuses on the science fiction novel...,SAKHAROV READINGS 2022: ENVIRONMENTAL PROBLEMS...,2022,POSTHUMANISM: ALTERNATIVE REALITIES AND AI IN ...,179,"[-0.11198905, 0.31442636, -0.8300437, -0.09335...",0.893007,The paper discusses the science fiction novels...


In [11]:
len(df)

20

In [12]:
answer_question(df, question=query, debug=False)


'Retrieval-Augmented Generation (RAG) is a method that extends beyond the pre-trained knowledge of Large Language Models (LLMs) by augmenting the original prompt with relevant passages or documents retrieved by an Information Retrieval (IR) system. RAG systems are increasingly important for Generative AI solutions, especially in scenarios where knowledge is constantly updated and cannot be memorized in the LLM. The retrieval component of RAG systems, whether dense or sparse, plays a crucial role in enhancing the effectiveness of LLMs. One key finding is that adding random documents in the prompt can improve the LLM accuracy by up to 35%. Additionally, the type of passages that IR systems within a RAG solution retrieve, their relevance to the query'